In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [82]:
# 유저 아이디(숫자형), 상품명(id,숫자형) 평점, 주문일자
data=pd.read_csv('상품협업필터링.csv')
data.head()

,상품등록일,상품명,상품코드,판매가,옵션입력,상품설명,평균평점
0,2021-07-28,로비 셔링 프릴나시,P0000MJX,29800,color{핑크|블랙},"['단독', '착용', '취향', '단독', '뷔', '스티', '도', '활용']",NaN
1,2021-07-28,미나 슬림핏 미니원피스,P0000MJW,19800,color{인디핑크|스카이블루|블루|블랙},"['미니', '꾸안꾸룩', '활용', '미니', '원피스', '하트', '넥', '...",NaN
2,2021-07-27,[La belle rose]퍼지 프릴 페이크 단추티,P0000MJV,16000,color{아이보리|베이지|핑크},"['프릴', '페이크', '단추', '걱정', '수', '실용', '만점', '아이템']",NaN
3,2021-07-27,"유니 세미 부츠컷팬츠(숏,롱.ver)",P0000MJU,23000,color{아이보리|블랙}//size{S/short.ver|M/short.ver|L...,"['숏', '롱', '두', '가지', '체형', '실루엣', '선택', '기장',...",NaN
4,2021-07-27,[La belle rose/1+1♥]도넌 심플 유넥 반팔티,P0000MJT,14000,color{아이보리|베이지|핑크|그린|블랙},"['기본', '더', '신경', '티셔츠', '장', '소장', '추천']",NaN


In [83]:
data.dropna(axis=0)
data.head()

,상품등록일,상품명,상품코드,판매가,옵션입력,상품설명,평균평점
0,2021-07-28,로비 셔링 프릴나시,P0000MJX,29800,color{핑크|블랙},"['단독', '착용', '취향', '단독', '뷔', '스티', '도', '활용']",NaN
1,2021-07-28,미나 슬림핏 미니원피스,P0000MJW,19800,color{인디핑크|스카이블루|블루|블랙},"['미니', '꾸안꾸룩', '활용', '미니', '원피스', '하트', '넥', '...",NaN
2,2021-07-27,[La belle rose]퍼지 프릴 페이크 단추티,P0000MJV,16000,color{아이보리|베이지|핑크},"['프릴', '페이크', '단추', '걱정', '수', '실용', '만점', '아이템']",NaN
3,2021-07-27,"유니 세미 부츠컷팬츠(숏,롱.ver)",P0000MJU,23000,color{아이보리|블랙}//size{S/short.ver|M/short.ver|L...,"['숏', '롱', '두', '가지', '체형', '실루엣', '선택', '기장',...",NaN
4,2021-07-27,[La belle rose/1+1♥]도넌 심플 유넥 반팔티,P0000MJT,14000,color{아이보리|베이지|핑크|그린|블랙},"['기본', '더', '신경', '티셔츠', '장', '소장', '추천']",NaN


In [84]:
data.loc[:,'상품코드'] = data.loc[:,'상품코드'].astype('category').cat.codes
data.loc[:,'상품등록일'] = data.loc[:,'상품등록일'].astype('category').cat.codes
data.loc[:,'상품명'] = data.loc[:,'상품명'].astype('category').cat.codes
data.loc[:,'옵션입력'] = data.loc[:,'옵션입력'].astype('category').cat.codes
data.loc[:,'상품설명'] = data.loc[:,'상품설명'].astype('category').cat.codes
data.head()

,상품등록일,상품명,상품코드,판매가,옵션입력,상품설명,평균평점
0,121,173,582,29800,337,70,NaN
1,121,264,581,19800,294,207,NaN
2,120,72,580,16000,187,544,NaN
3,120,444,579,23000,197,298,NaN
4,120,13,578,14000,176,21,NaN


In [85]:
data1=data.pivot_table('평균평점',index='상품코드',columns='판매가').fillna(0)

In [86]:
data1.head()

판매가,7000,9000,10000,11000,11800,12000,13000,14000,15000,16000,...,58800,59500,59800,61000,65000,66500,68000,72000,76000,84000
상품코드,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
data1.shape

(455, 89)

In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   상품등록일   583 non-null    int8   
 1   상품명     583 non-null    int16  
 2   상품코드    583 non-null    int16  
 3   판매가     583 non-null    int64  
 4   옵션입력    583 non-null    int16  
 5   상품설명    583 non-null    int16  
 6   평균평점    455 non-null    float64
dtypes: float64(1), int16(4), int64(1), int8(1)
memory usage: 14.4 KB


In [89]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 455 entries, 0 to 568
Data columns (total 89 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   7000    455 non-null    float64
 1   9000    455 non-null    float64
 2   10000   455 non-null    float64
 3   11000   455 non-null    float64
 4   11800   455 non-null    float64
 5   12000   455 non-null    float64
 6   13000   455 non-null    float64
 7   14000   455 non-null    float64
 8   15000   455 non-null    float64
 9   16000   455 non-null    float64
 10  17000   455 non-null    float64
 11  18000   455 non-null    float64
 12  18500   455 non-null    float64
 13  18800   455 non-null    float64
 14  19000   455 non-null    float64
 15  19500   455 non-null    float64
 16  19800   455 non-null    float64
 17  20000   455 non-null    float64
 18  20500   455 non-null    float64
 19  20800   455 non-null    float64
 20  21000   455 non-null    float64
 21  22000   455 non-null    float64
 22  22

In [90]:
product_sim=cosine_similarity(data,data1)
print(product_sim.shape)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').